In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import csv
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import concurrent.futures


In [8]:
main_url = 'https://www.iranketab.ir/book'

In [9]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}


In [10]:
""" response = requests.get(url=main_url , headers=headers)
print(response.status_code) """

' response = requests.get(url=main_url , headers=headers)\nprint(response.status_code) '

In [11]:
""" soup = BeautifulSoup(response.content, 'html.parser') """

" soup = BeautifulSoup(response.content, 'html.parser') "

In [12]:
""" page_number = 1  # Start with the first page

request_count = 0

session = requests.Session()
visited_urls = set()

with open('BooksUrls.csv', 'w', newline='') as csvfile:
    url_writer = csv.writer(csvfile)

    while True:
        current_page_url = f"{main_url}?pagenumber={page_number}&pagesize=20"
        response = session.get(current_page_url)

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        allBooks = soup.find_all("a", class_="product-item-link")
        
        for book_url in allBooks:
            href = book_url.get("href")
            if href:
                full_url = f'{main_url}{href}'
                if full_url not in visited_urls:
                    visited_urls.add(full_url)
                    url_writer.writerow([full_url])
                    request_count += 1

                    if request_count % 500 == 0:
                        time.sleep(1)

        next_page_link = soup.find("a", {"data-page-no": str(page_number + 1)})

        if next_page_link:
            page_number += 1
        else:
            break """

' page_number = 1  # Start with the first page\n\nrequest_count = 0\n\nsession = requests.Session()\nvisited_urls = set()\n\nwith open(\'BooksUrls.csv\', \'w\', newline=\'\') as csvfile:\n    url_writer = csv.writer(csvfile)\n\n    while True:\n        current_page_url = f"{main_url}?pagenumber={page_number}&pagesize=20"\n        response = session.get(current_page_url)\n\n        if response.status_code != 200:\n            break\n\n        soup = BeautifulSoup(response.text, \'html.parser\')\n        allBooks = soup.find_all("a", class_="product-item-link")\n        \n        for book_url in allBooks:\n            href = book_url.get("href")\n            if href:\n                full_url = f\'{main_url}{href}\'\n                if full_url not in visited_urls:\n                    visited_urls.add(full_url)\n                    url_writer.writerow([full_url])\n                    request_count += 1\n\n                    if request_count % 500 == 0:\n                        time.sle

In [13]:
#csv_file = 'final_links.csv'

In [14]:
""" BookURLs = pd.read_csv('BooksUrls.csv', header=None)
urls = BookURLs[0].tolist()
urls = [url.replace("/book", "", 1) for url in urls]
link = pd.DataFrame({'final_books_Url': urls}) """

' BookURLs = pd.read_csv(\'BooksUrls.csv\', header=None)\nurls = BookURLs[0].tolist()\nurls = [url.replace("/book", "", 1) for url in urls]\nlink = pd.DataFrame({\'final_books_Url\': urls}) '

In [15]:
#link.to_csv('final_books_Url.csv', index=False)

In [16]:
BookURLs = pd.read_csv('final_books_Url.csv', header=None)
links = BookURLs[0].tolist()

In [17]:
retry_strategy = Retry(
    total=100,
    backoff_factor=2,
    status_forcelist=[500, 502, 503, 504],
)

session = requests.Session()
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

csv_file = 'scraped_data5.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.DictWriter(file, fieldnames=[
        'book_id', 'print_series', 'size', 'type_of_print', 'translator',
        'shabak', 'gregorian_publish_year', 'solar_publish_year', 'page_count',
        'Persian Title', 'English Title', 'Off', 'Rate', 'Break Price', 'Special_Price',
        'Exist', 'Publisher', 'publisher id', 'Writer', 'writer id', 'Description', 'Feature', 'Category'
    ])

    csv_writer.writeheader()

    session = requests.Session()
    request_count = 0

    for link in links[50000:]:
        try:
            response = session.get(link)
            if response.status_code == 200:
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                clearfix_elements = soup.find_all('div', class_='clearfix')
                writer_id = None
                publisher_id = None
                for clearfix in clearfix_elements:
                    product_name_element = clearfix.find(class_='product-name')
                    if product_name_element:
                        persian_title = product_name_element.text.strip()

                        en_name_element = clearfix.find('div', class_='product-name-englishname ltr')
                        if en_name_element:
                            english_title = en_name_element.text.strip()
                        else:
                            english_title = ""

                        off = clearfix.find('div', style='float: left;font-size: 12px;line-height: 1.375;background-color: #fb3449;color: #fff;padding: 5px 30px 3px;-webkit-border-radius: 0 16px 16px 16px;border-radius: 0 16px 16px 16px;')
                        if off:
                            off_percent = off.text.strip()
                        else:
                            off_percent = 0
                        rating_div = clearfix.find('div', class_='my-rating')
                        if rating_div:
                            data_rating = rating_div.get('data-rating')
                        else:
                            data_rating = 0
                        break_price = clearfix.find('span' , class_= 'price price-broken')
                        if break_price:
                            before_price = break_price.text.strip()
                        else :
                            before_price = np.nan
                        special_price = clearfix.find('span' , class_= 'price price-special')
                        if special_price:
                            after_price = special_price.text.strip()
                        else :
                            after_price = np.nan
                        exists_book_element = clearfix.find('li', class_='exists-book')
                        if exists_book_element:
                            exists_book_text = exists_book_element.text.strip()
                        else:
                            exists_book_text = 'ناموجود'
                        publisher_element = clearfix.find('div', class_='col-xs-12 prodoct-attribute-items')
                        if publisher_element:
                            publisher_span = publisher_element.find('span', class_='prodoct-attribute-item')
                            if publisher_span and publisher_span.text.strip() == 'انتشارات:':
                                publisher_text = publisher_element.find('span', class_='prodoct-attribute-item').find_next('span').text.strip()
                            else:
                                publisher_text = ""
                        else:
                            publisher_text = ""
                        #########################################################################################################################################
                        look_for_id = clearfix.find('div', class_='row clearfix')
                        for id_publisher_and_writer in look_for_id.find_all('a'):
                            if id_publisher_and_writer:
                                href = id_publisher_and_writer.get('href')
                                if 'publisher' in href:
                                    publisher_id = int(href.split('/publisher/')[1].split('-')[0])
                                elif 'profile' in href:
                                    writer_id = int(href.split('/profile/')[1].split('-')[0])
                        #########################################################################################################################################
                        writer_element = clearfix.find('span', itemprop='name')
                        if writer_element:
                            writer_text = writer_element.text.strip()
                        else:
                            writer_text = ""

                        description_element = soup.find('div', class_='product-description')
                        if description_element:
                            description_text = description_element.text.strip()
                        else:
                            description_text = ""
                        feature_elements = soup.find_all('div', class_='product-features')
                        features = []
                        for feature_element in feature_elements:
                            h4_elements = feature_element.find_all('h4')
                            for h4_element in h4_elements:
                                feature_text = h4_element.text.strip()
                                features.append(feature_text)
                        product_tags_div = soup.find('div', class_='product-tags')
                        if product_tags_div:
                            h5_elements = product_tags_div.find_all('h5')
                            tags = [h5.text.strip() for h5 in h5_elements]
                        else:
                            tags = []
                        book_id = None
                        shabak = None
                        page_count = None
                        solar_publish_year = None
                        gregorian_publish_year = None
                        print_series = None
                        type_of_print = None
                        size = None
                        translator_info = {}
                        product_info = clearfix.find('table', class_='product-table')
                        if product_info:
                            attr_elements = product_info.find_all('td')
                            for i in range(0, len(attr_elements), 2):
                                attr_name = attr_elements[i].text.strip()
                                attr_value = attr_elements[i + 1].text.strip()
                                if attr_name == 'کد کتاب :':
                                    book_id = attr_value
                                
                                if 'مترجم' in attr_name and attr_value:
                                    translator_elements = attr_elements[i + 1].find_all('a', itemprop='author')
                                    for translator_element in translator_elements:
                                        translator_id = int(translator_element.get('href').split('/profile/')[1].split('-')[0])
                                        translator_name = translator_element.find('span', itemprop='name').text.strip()
                                        translator_info[translator_name] = translator_id
                                if 'شابک' in attr_name:
                                    shabak = attr_value
                                if 'تعداد صفحه' in attr_name:
                                    page_count = attr_value
                                if 'سال انتشار شمسی' in attr_name:
                                    solar_publish_year = attr_value
                                if 'سال انتشار میلادی' in attr_name:
                                    gregorian_publish_year = attr_value
                                if 'نوع جلد' in attr_name:
                                    type_of_print = attr_value
                                if 'سری چاپ' in attr_name:
                                    print_series = attr_value
                                if 'قطع' in attr_name:
                                    size = attr_value
                        scraped_data = {
                            'book_id': book_id,
                            'print_series': print_series,
                            'size': size,
                            'type_of_print': type_of_print,
                            'translator': translator_info,
                            'shabak': shabak,
                            'gregorian_publish_year': gregorian_publish_year,
                            'solar_publish_year': solar_publish_year,
                            'page_count': page_count,
                            'Persian Title': persian_title,
                            'English Title': english_title,
                            'Off': off_percent,
                            'Rate': round(float(data_rating), 2),
                            'Break Price': before_price,
                            'Special_Price': after_price,
                            'Exist': exists_book_text,
                            'Publisher': publisher_text,
                            'publisher id': publisher_id,
                            'Writer': writer_text,
                            'writer id': writer_id,
                            'Description': description_text,
                            'Feature': features,
                            'Category': tags
                        }
                        csv_writer.writerow(scraped_data)
                        
                    request_count += 1
                    if request_count % 500 == 0:
                        time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for link: {link}")
            print(e)
            continue 
session.close()

Request failed for link: https://www.iranketab.ir/book/87210-
Exceeded 30 redirects.
Request failed for link: https://www.iranketab.ir/book/90319-
Exceeded 30 redirects.
Request failed for link: https://www.iranketab.ir/book/90591-
Exceeded 30 redirects.
Request failed for link: https://www.iranketab.ir/book/120654-
Exceeded 30 redirects.


In [19]:
data1 = pd.read_csv('scraped_data4.csv')
data2 = pd.read_csv('scraped_data5.csv')

C:\Users\BAAZSHOW\AppData\Local\Temp\ipykernel_10128\1251195954.py:1: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv('scraped_data4.csv')
C:\Users\BAAZSHOW\AppData\Local\Temp\ipykernel_10128\1251195954.py:2: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv('scraped_data5.csv')


In [20]:
book1 = pd.DataFrame(data1)
book2 = pd.DataFrame(data2)
book = pd.concat([book1, book2], axis=0)

In [24]:
book.drop_duplicates(subset=['Persian Title', 'English Title' , 'Off'	,'Rate'	,'Break Price',	'Special_Price'	,'Exist',	'Publisher'	,'Writer'], inplace=True)

In [199]:
book.to_csv('info_book.csv', index=False)

## CLEANING DATA

In [214]:
data_cleaning = pd.read_csv('info_book.csv')

C:\Users\BAAZSHOW\AppData\Local\Temp\ipykernel_10128\1131713321.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cleaning = pd.read_csv('info_book.csv')


In [215]:
data_cleaning.dropna(subset=['shabak' , 'book_id' , 'publisher id' , 'writer id' , 'Writer'] , inplace=True)

In [216]:
data_cleaning['translator'].replace("{}", np.nan, inplace=True)
data_cleaning['Feature'].replace("[]", np.nan, inplace=True)
data_cleaning['translator'].fillna('Unknown',inplace=True)
data_cleaning['gregorian_publish_year'].fillna('Unknown',inplace=True)
data_cleaning['solar_publish_year'].fillna('Unknown',inplace=True)
data_cleaning['page_count'].fillna('Unknown',inplace=True)
data_cleaning['English Title'].fillna('Unknown',inplace=True)
data_cleaning['Description'].fillna('Not Define',inplace=True)
data_cleaning['Feature'].fillna('Not Define',inplace=True)
data_cleaning['print_series'].fillna('Unknown',inplace=True)
data_cleaning['size'].fillna('Not Define',inplace=True)
data_cleaning['type_of_print'].fillna('Not Define',inplace=True)
data_cleaning['Break Price'].fillna('Not Define',inplace=True)
data_cleaning['Special_Price'].fillna('Not Define',inplace=True)
data_cleaning['Off'] = data_cleaning['Off'].str.replace('% تخفیف', '', regex=False)

In [217]:
def fix_isbn(isbn):
    isbn = ''.join(filter(str.isdigit, str(isbn)))
    if len(isbn) == 10:
        isbn = '978' + isbn
    if len(isbn) == 13:
        formatted_isbn = f'{isbn[:3]}-{isbn[3:12]}-{isbn[12]}'
        return formatted_isbn
    return isbn
data_cleaning['shabak'] = data_cleaning['shabak'].apply(fix_isbn)

In [218]:
def clean_solar_year(solar_year):
    try:
        year_match = re.search(r'\d+', str(solar_year))
        if year_match:
            cleaned_year = int(year_match.group())
            if 1000 <= cleaned_year <= 1500:  
                return cleaned_year
    except (ValueError, TypeError):
        pass
    return np.nan
data_cleaning['solar_publish_year'] = data_cleaning['solar_publish_year'].apply(clean_solar_year)

def clean_gregorian_year(gregorian_year):
    try:
        cleaned_year = re.sub(r'[^\d-]', '', str(gregorian_year))
        
        year_match = re.search(r'\d+', cleaned_year)
        if year_match:
            cleaned_year = int(year_match.group())
            
            if 1000 <= cleaned_year <= 3000:
                return cleaned_year
    except (ValueError, TypeError):
        pass
    
    return np.nan
data_cleaning['gregorian_publish_year'] = data_cleaning['gregorian_publish_year'].apply(clean_gregorian_year)



In [219]:
data_cleaning['gregorian_publish_year'] = data_cleaning['gregorian_publish_year'].fillna(0).astype(int)
data_cleaning['solar_publish_year'] = data_cleaning['solar_publish_year'].fillna(0).astype(int)
data_cleaning['solar_publish_year'] = data_cleaning['solar_publish_year'].replace(0, 'unknown')
data_cleaning['gregorian_publish_year'] = data_cleaning['gregorian_publish_year'].replace(0, 'unknown')


In [220]:
def convert_to_int(value):
    try:
        return int(float(value))
    except (ValueError, TypeError):
        return 0
data_cleaning['print_series'] = data_cleaning['print_series'].apply(convert_to_int)
data_cleaning['publisher id'] = data_cleaning['publisher id'].apply(convert_to_int)
data_cleaning['writer id'] = data_cleaning['writer id'].apply(convert_to_int)
data_cleaning['book_id'] = data_cleaning['book_id'].apply(convert_to_int)


In [221]:
data_cleaning['Break Price'] = data_cleaning['Break Price'].str.replace(',', '', regex=True)
data_cleaning['Special_Price'] = data_cleaning['Special_Price'].str.replace(',', '', regex=True)

In [222]:
data_cleaning.head()

,book_id,print_series,size,type_of_print,translator,shabak,gregorian_publish_year,solar_publish_year,page_count,Persian Title,...,Break Price,Special_Price,Exist,Publisher,publisher id,Writer,writer id,Description,Feature,Category
0,71718,80,رقعی,جلد سخت,{'پیمان خاکسار': 164},978-600229500-2,2008,1402,779,کتاب جزء از کل (مجموعه برگ و نوا),...,390000,292500,موجود,نشر چشمه,33,استیو تولتز,27,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...,['برنده جایزه ی NSW Premier سال 2009'],"['جایزه ی ان اس دبلیو پرایمر', 'ادبیات استرالی..."
1,92408,4,رقعی,جلد سخت,{'گلناز سهرابی': 49301},978-600989648-6,2008,1401,544,جزء از کل,...,420000,252000,موجود,ندای معاصر (زرین کلک),1869,استیو تولتز,27,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...,['برنده جایزه ی NSW Premier سال 2009'],"['جایزه ی ان اس دبلیو پرایمر', 'ادبیات استرالی..."
2,92404,4,رقعی,شومیز,{'گلناز سهرابی': 49301},978-600989640-0,2008,1401,544,جزء از کل,...,500000,300000,موجود,ندای معاصر (زرین کلک),1869,استیو تولتز,27,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...,['برنده جایزه ی NSW Premier سال 2009'],"['جایزه ی ان اس دبلیو پرایمر', 'ادبیات استرالی..."
3,43,80,رقعی,جلد سخت,{'پیمان خاکسار': 164},978-600229500-2,2008,1402,656,جزء از کل,...,Not Define,Not Define,ناموجود,نشر چشمه,33,استیو تولتز,27,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...,['برنده جایزه ی NSW Premier سال 2009'],"['جایزه ی ان اس دبلیو پرایمر', 'ادبیات استرالی..."
4,2030,18,رقعی,شومیز,{'مهدی غبرایی': 16},978-964448297-7,2003,1400,368,کتاب بادبادک باز,...,165000,140250,موجود,نیلوفر,67,خالد حسینی,74,امیر، پسر یک تاجر ثروتمند کابلی و از قبیله ی ص...,"['برنده جایزه بوکه سال 2004', 'در سال ۲۰۰۷ میل...","['ادبیات اقتباسی', 'جایزه ی الکس', 'جایزه ی بو..."


In [223]:
data_cleaning.isna().sum()

book_id                   0
print_series              0
size                      0
type_of_print             0
translator                0
shabak                    0
gregorian_publish_year    0
solar_publish_year        0
page_count                0
Persian Title             0
English Title             0
Off                       0
Rate                      0
Break Price               0
Special_Price             0
Exist                     0
Publisher                 0
publisher id              0
Writer                    0
writer id                 0
Description               0
Feature                   0
Category                  0
dtype: int64

In [224]:
data_cleaning.to_csv('Book.csv', index=False)